In [59]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

tfidf_word_value = []
X = []

readIn = pd.read_csv("../Clean/lemma_result.csv")

X_temp = readIn.iloc[:500, :-1].values
Y = readIn.iloc[:500, -1].values

for sentence in X_temp:
    X.append(sentence[0])

#split data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

# TF-IDF
vectorizer = TfidfVectorizer()
df_train = vectorizer.fit_transform(X_train).toarray()
df_test = vectorizer.transform(X_test).toarray()
print(df_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [7]:
print(len(vectorizer.vocabulary_))

477


In [44]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
vocab_size = len(vectorizer.vocabulary_)
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,     
                                     input_length=vocab_size) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 9218, 32)          294976    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 9218, 32)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                16600     
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 311,627
Trainable params: 311,627
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
history = model.fit(df_train,Y_train,
                  validation_split=0.2, epochs=5, batch_size=32)

Train on 320 samples, validate on 80 samples
Epoch 1/5
320/320 [==============================] - 224s 700ms/sample - loss: 0.6944 - accuracy: 0.4781 - val_loss: 0.6922 - val_accuracy: 0.5500
Epoch 2/5
320/320 [==============================] - 241s 754ms/sample - loss: 0.6948 - accuracy: 0.4781 - val_loss: 0.6918 - val_accuracy: 0.5500
Epoch 3/5
320/320 [==============================] - 224s 701ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6916 - val_accuracy: 0.5500
Epoch 4/5
320/320 [==============================] - 228s 712ms/sample - loss: 0.6934 - accuracy: 0.4938 - val_loss: 0.6912 - val_accuracy: 0.5500
Epoch 5/5
320/320 [==============================] - 227s 711ms/sample - loss: 0.6927 - accuracy: 0.5000 - val_loss: 0.6914 - val_accuracy: 0.5500


In [61]:
from tensorflow.keras.models import model_from_yaml

# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
